## Example: BX610 (High-z SFG)
### Prepare visibiity data from the calibrated MS restored by the local ALMA pipeline

#### Load some essential Python modules + CASA 6 tasks/tools 

In [1]:

import sys,os,glob,io,socket
import logging
from pprint import pprint
import numpy as np

import casatasks as ctasks
import casatools as ctools

# Import wurlitzer for display real-time console logs
#   https://github.com/minrk/wurlitzer
%reload_ext wurlitzer

# for inline plots
%matplotlib inline
%config InlineBackend.figure_format = "retina"


Using the beta-version of CASA 6

In [2]:
print('casatools ver:',ctools.version_string())
print('casatasks ver:',ctasks.version_string())

casatools ver: 2019.172
casatasks ver: 2019.166


#### Import some convinient functions from `rxutils`

In [3]:
from rxutils.casa.proc import rmPointing        # help remove POINTING tables hidden under
from rxutils.casa.proc import setLogfile        # help reset the casa 6 log file
from rxutils.casa.proc import checkchflag       # help check channel-wise flagging stats
from rxutils.casa.proc import getcommonfreqs    # obatin the frequency coverage of one SPW in a specified frame (e.g. TOPO-LSRK)

### 2013.1.00059.S (Band 4, Cycle-2, Aravena M.) 

#### Spws Setup

+ spw=0  BB1  153GHz   CI 1-0
+ spw=1  BB2  155GHz   Continuum
+ spw=2  BB3  143GHz   CO 4-3
+ spw=3  BB4  141GHz   Continuum

Beam: 0.35"x0.30"

#### Export one MS per baseband

+ Combine the SPWs from each baseband into a single SPW
+ Transform the frame from TOPO to LSRK
+ Exclude the edge channels

Note: GMaKE requires the input MS contains only one SPW and one FIELD.

#### MS manupilation

+ The integration time is rebined to 60s (limited by the time-averging smearing)
+ The continuum SPW is averged into 1 representive channel (limited by the bandwidth smearing)
+ The point tables were removed[<sup>1</sup>](#fn1): they are only used for mosaic imaging, not in this specific case; for ALMA data, the table can be quite large even for channle-/time-averaged data)

<span id="fn1"> <sup>1</sup></span> We can't remove ms/POINTING table using `rmtables()` as its abence will crash certain CASA tasks/tools (e.g. listable); instead, we set it to have empty row, as it's done in `rmPointing()`.

In [10]:

# Switch working directory

demo_dir='/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/data/bx610/alma/2013.1.00059.S/'
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir(demo_dir)
setLogfile(demo_dir+'/'+'demo_bx610.log')

vis_name='/Volumes/D1/projects/hzdyn/2013.1.00059.S/science_goal.uid___A001_X12b_X239/group.uid___A001_X12b_X23a/member.uid___A001_X12b_X23b/calibrated/uid___A001_X12b_X23c_target.ms'
#ctasks.listobs(vis_name)


spw_list=['0','1','2','3']
bb_list=['1','2','3','4']
field='BX610'

# For BB1 & BB2 & BB3 & BB4: TOPO->LSKR

"""
for i in range(0,4):
    outvis='bb'+bb_list[i]+'.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform(vis_name,outputvis=outvis,field='BX610',spw=spw_list[i],datacolumn='data',
                        regridms=True,outframe='lsrk',combinespws=True,mode='channel',start=1,nchan=478,width=1,
                        timeaverage=True,timebin='60s',maxuvwdistance=0.0,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)

"""

# For BB2/BB4: Channel-averging

for i in range(0,4):
    if  bb_list[i]=='1' or bb_list[i]=='3':
        continue
    outvis='bb'+bb_list[i]+'.mfs.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform('bb'+bb_list[i]+'.ms',outputvis=outvis,field='BX610',datacolumn='data',
                        chanaverage=True,chanbin=478,
                        timeaverage=False,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)


2019-10-14 16:37:38	INFO	mstransform::::casa	##########################################
2019-10-14 16:37:38	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-14 16:37:38	INFO	mstransform::::casa	mstransform( vis='bb2.ms', outputvis='bb2.mfs.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=False, chanaverage=True, chanbin=478, hanning=False, regridms=False, mode='channel', nchan=-1, start=0, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='', veltype='radio', preaverage=False, timeaverage=False, timebin='0s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, want_cont=False, denoising_lib=True, nthreads=1, niter=1, disableparallel=Fals

 40M	bb2.mfs.ms


2019-10-14 16:37:49	INFO	mstransform::::casa	##########################################
2019-10-14 16:37:49	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-14 16:37:49	INFO	mstransform::::casa	mstransform( vis='bb4.ms', outputvis='bb4.mfs.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=False, chanaverage=True, chanbin=478, hanning=False, regridms=False, mode='channel', nchan=-1, start=0, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='', veltype='radio', preaverage=False, timeaverage=False, timebin='0s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, want_cont=False, denoising_lib=True, nthreads=1, niter=1, disableparallel=Fals

 40M	bb4.mfs.ms


2019-10-14 16:37:59	INFO	mstransform::::casa	Result mstransform: True
2019-10-14 16:37:59	INFO	mstransform::::casa	Task mstransform complete. Start time: 2019-10-14 11:37:49.137738 End time: 2019-10-14 11:37:59.379531
2019-10-14 16:37:59	INFO	mstransform::::casa	##### End Task: mstransform          #####
2019-10-14 16:37:59	INFO	mstransform::::casa	##########################################


### 2015.1.00250.S (Band 6, Cycle-3, Aravena M.) 

#### Spws Setup

+ spw=0 BB1    250GHz   Continuum
+ spw=1 BB2    252GHz   CO 7-6 / CI 2-1
+ spw=2 BB3    234GHz   H2O
+ spw=3 BB4    236GHz   Continuum

Beam: 0.27"x0.24"

In [11]:
# Switch working directory

demo_dir='/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/data/bx610/alma/2015.1.00250.S/'
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir(demo_dir)
setLogfile(demo_dir+'/'+'demo_bx610.log')

vis_name='/Volumes/D1/projects/hzdyn/2015.1.00250.S/science_goal.uid___A001_X2fe_X20d/group.uid___A001_X2fe_X20e/member.uid___A001_X2fe_X20f/calibrated/uid___A001_X2fe_X20f_target.ms'
#ctasks.listobs(vis_name)


spw_list=['0','1','2','3']
bb_list=['1','2','3','4']
field='BX610'

# For BB1 & BB2 & BB3 & BB4: TOPO->LSKR


for i in range(0,4):
    outvis='bb'+bb_list[i]+'.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform(vis_name,outputvis=outvis,field='BX610',spw=spw_list[i],datacolumn='data',
                        regridms=True,outframe='lsrk',combinespws=True,mode='channel',start=1,nchan=238,width=1,
                        timeaverage=True,timebin='60s',maxuvwdistance=0.0,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)
    
# For BB1/BB4: Channel-averging

for i in range(0,4):
    if  bb_list[i]=='2' or bb_list[i]=='3':
        continue
    outvis='bb'+bb_list[i]+'.mfs.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform('bb'+bb_list[i]+'.ms',outputvis=outvis,field='BX610',datacolumn='data',
                        chanaverage=True,chanbin=238,
                        timeaverage=False,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)    


2019-10-14 16:44:11	INFO	mstransform::::casa	##########################################
2019-10-14 16:44:11	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-14 16:44:11	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/projects/hzdyn/2015.1.00250.S/science_goal.uid___A001_X2fe_X20d/group.uid___A001_X2fe_X20e/member.uid___A001_X2fe_X20f/calibrated/uid___A001_X2fe_X20f_target.ms', outputvis='bb1.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='0', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='channel', nchan=238, start=1, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='lsrk', veltype='radio', preaverage=False, timeaverage=True, timebin='60s', timespan='', ma

245M	bb1.ms


2019-10-14 16:45:24	INFO	mstransform::::casa	##########################################
2019-10-14 16:45:24	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-14 16:45:24	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/projects/hzdyn/2015.1.00250.S/science_goal.uid___A001_X2fe_X20d/group.uid___A001_X2fe_X20e/member.uid___A001_X2fe_X20f/calibrated/uid___A001_X2fe_X20f_target.ms', outputvis='bb2.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='1', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='channel', nchan=238, start=1, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='lsrk', veltype='radio', preaverage=False, timeaverage=True, timebin='60s', timespan='', ma

245M	bb2.ms


2019-10-14 16:46:37	INFO	mstransform::::casa	##########################################
2019-10-14 16:46:37	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-14 16:46:37	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/projects/hzdyn/2015.1.00250.S/science_goal.uid___A001_X2fe_X20d/group.uid___A001_X2fe_X20e/member.uid___A001_X2fe_X20f/calibrated/uid___A001_X2fe_X20f_target.ms', outputvis='bb3.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='2', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='channel', nchan=238, start=1, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='lsrk', veltype='radio', preaverage=False, timeaverage=True, timebin='60s', timespan='', ma

...

245M	bb4.ms


2019-10-14 16:49:42	INFO	mstransform::::casa	##########################################
2019-10-14 16:49:42	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-14 16:49:42	INFO	mstransform::::casa	mstransform( vis='bb1.ms', outputvis='bb1.mfs.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=False, chanaverage=True, chanbin=238, hanning=False, regridms=False, mode='channel', nchan=-1, start=0, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='', veltype='radio', preaverage=False, timeaverage=False, timebin='0s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, want_cont=False, denoising_lib=True, nthreads=1, niter=1, disableparallel=Fals

 32M	bb1.mfs.ms


2019-10-14 16:49:50	INFO	mstransform::::casa	##########################################
2019-10-14 16:49:50	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-14 16:49:50	INFO	mstransform::::casa	mstransform( vis='bb4.ms', outputvis='bb4.mfs.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=False, chanaverage=True, chanbin=238, hanning=False, regridms=False, mode='channel', nchan=-1, start=0, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='', veltype='radio', preaverage=False, timeaverage=False, timebin='0s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, want_cont=False, denoising_lib=True, nthreads=1, niter=1, disableparallel=Fals

 32M	bb4.mfs.ms


### 2017.1.01045.S (Band 4, Cycle-5, Brisbin, L.) 

#### Spws Setup

+ spw=25/0 BB=1    143GHz   CO 4-3 
+ spw=27/1 BB=2    141GHz   Continuum
+ spw=29/2 BB=3    153GHz   CI 1-0
+ spw=31/3 BB=4    155GHz   Continuum

Beam: 0.02"x0.02"

#### Export one MS per baseband

+ Combine the SPWs from each baseband into a single SPW
+ Transform the frame from TOPO to LSRK
+ Exclude the edge channels

Note: GMaKE requires the input MS contains only one SPW and one FIELD.

#### MS manupilation

+ The integration time is rebined to 60s (limited by the time-averging smearing)
+ The continuum SPW is averged into 1 representive channel (limited by the bandwidth smearing)
+ The point tables were removed[<sup>1</sup>](#fn1): they are only used for mosaic imaging, not in this specific case; for ALMA data, the table can be quite large even for channle-/time-averaged data)

<span id="fn1"> <sup>1</sup></span> We can't remove ms/POINTING table using `rmtables()` as its abence will crash certain CASA tasks/tools (e.g. listable); instead, we set it to have empty row, as it's done in `rmPointing()`.

In [4]:

demo_dir='/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/data/bx610/alma/2017.1.01045.S/'
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir(demo_dir)
setLogfile(demo_dir+'/'+'demo_bx610.log')

repo='/Volumes/D1/proc/2017.1.01045.S/science_goal.uid___A001_X1288_X10c4/group.uid___A001_X1288_X10c5/member.uid___A001_X1288_X10c6/calibrated/'
uid_list=['uid___A002_Xc69057_X91a','uid___A002_Xc69057_Xc5c','uid___A002_Xc69057_X1293','uid___A002_Xc6a3db_X430','uid___A002_Xc6c0d5_X57ea']
spw_list=['25','27','29','31']
bb_list=['1','2','3','4']
field='BX610'

# For BB1 & BB2 & BB3 & BB4: TOPO->LSKR

for i in range(0,4):


    freqs,chanwidth=getcommonfreqs([repo+uid+'.ms' for uid in uid_list],
                                   [spw_list[i]]*len(uid_list),
                                   chanbin=4,frame='LSRK',edge=1)
    print(min(freqs),max(freqs),chanwidth,len(freqs))
    
    """
    if  i==0:
        freq1='142633992676.3963'
        freq2=479
        freq3='3906451.4650878906'
    if  i==1:
        freq1='140805534867.07092'
        freq2=479
        freq3='3906451.4650878906'
    if  i==2:
        freq1='152676489541.2054'
        freq2=479
        freq3='3906451.4653320312'
    if  i==3:
        freq1='154508695285.6513'
        freq2=479
        freq3='3906451.4650878906'        
    """
    
    vis_list=[]
    for uid in uid_list:
        vis_name=repo+uid+'.ms'
        
        outvis=uid+'_bb'+bb_list[i]+'.ms'
        
 
        os.system('rm -rf '+outvis)
        # use mode='freuqnecy' so if the basebands from different sessions are slightly offseted in TOPO, the concat result will still contain a single LSRK spw
        
        ctasks.mstransform(vis_name,outputvis=outvis,field='BX610',spw=spw_list[i],datacolumn='corrected',
                            regridms=True,outframe='lsrk',combinespws=True,mode='frequency',\
                            start=str(min(freqs))+'Hz',nchan=len(freqs),width=str(chanwidth)+'Hz',
                            timeaverage=True,timebin='60s',maxuvwdistance=0.0,
                            keepflags=False,usewtspectrum=False,intent='OBSERVE_TARGET#ON_SOURCE')          
        rmPointing(outvis,verbose=True)

        vis_list+=[outvis]

    os.system('rm -rf '+'bb'+bb_list[i]+'.ms')
    ctasks.concat(vis=vis_list,concatvis='bb'+bb_list[i]+'.ms',copypointing=True)
    ctasks.listobs('bb'+bb_list[i]+'.ms')
    checkchflag('bb'+bb_list[i]+'.ms')

# For BB2/BB4: Channel-averging

for i in range(0,4):
    if  bb_list[i]=='1' or bb_list[i]=='3':
        continue
    if  bb_list[i]=='1' or bb_list[i]=='2':
        spw='0:0~477'
    else:
        spw='0:1~478'
    outvis='bb'+bb_list[i]+'.mfs.ms'
    os.system('rm -rf '+outvis)
    ctasks.mstransform('bb'+bb_list[i]+'.ms',outputvis=outvis,field='BX610',datacolumn='data',
                        chanaverage=True,chanbin=478,spw='0:0~477',
                        timeaverage=False,
                        keepflags=False,usewtspectrum=False)
    rmPointing(outvis)


##########



2019-10-26 14:31:34	INFO	ms::cvelfreqs	Calculating grid ...
2019-10-26 14:31:35	INFO	ms::cvelfreqs	Using observation time from earliest row of the MS given the SPW and FIELD selection:
2019-10-26 14:31:35	INFO	ms::cvelfreqs	    2017/11/06/23:02:01 (UTC)
2019-10-26 14:31:35	INFO	ms::cvelfreqs	Using tabulated observatory position for ALMA:
2019-10-26 14:31:35	INFO	ms::cvelfreqs	   Position: [2.22514e+06, -5.44031e+06, -2.48103e+06] (ITRF)
2019-10-26 14:31:35	INFO	SubMS::convertGridPars	 *** Encountered negative channel widths in input spectral window.
2019-10-26 14:31:35	INFO	SubMS::convertGridPars	 Channels equidistant in freq
2019-10-26 14:31:35	INFO	SubMS::convertGridPars+	 Central frequency (in output frame) = 1.43569e+11 Hz
2019-10-26 14:31:35	INFO	SubMS::convertGridPars+	 Channel central frequency is decreasing with increasing channel number.
2019-10-26 14:31:35	INFO	SubMS::convertGridPars+	 Width of central channel (in output frame) = 976610 Hz
2019-10-26 14:31:35	INFO	SubMS::conv

142633992684.47223 144501284205.37115 3906467.6169433594 479


2019-10-26 14:31:40	INFO	ParallelDataHelper::::casa	Parse time averaging parameters
2019-10-26 14:31:40	INFO	MSTransformManager::parseMsSpecParams	Input file name is /Volumes/D1/proc/2017.1.01045.S/science_goal.uid___A001_X1288_X10c4/group.uid___A001_X1288_X10c5/member.uid___A001_X1288_X10c6/calibrated/uid___A002_Xc69057_X91a.ms
2019-10-26 14:31:40	INFO	MSTransformManager::parseMsSpecParams	Data column is CORRECTED
2019-10-26 14:31:40	INFO	MSTransformManager::parseMsSpecParams	Output file name is uid___A002_Xc69057_X91a_bb1.ms
2019-10-26 14:31:40	INFO	MSTransformManager::parseMsSpecParams	Re-index is enabled 
2019-10-26 14:31:40	INFO	MSTransformManager::parseMsSpecParams	Tile shape is [0]
2019-10-26 14:31:40	INFO	MSTransformManager::parseDataSelParams	field selection is BX610
2019-10-26 14:31:40	INFO	MSTransformManager::parseDataSelParams	spw selection is 25
2019-10-26 14:31:40	INFO	MSTransformManager::parseDataSelParams	scan intent selection is OBSERVE_TARGET#ON_SOURCE
2019-10-26 14:3

Before rmPointing()
1.2G	uid___A002_Xc69057_X91a_bb1.ms
After rmPointing()
434M	uid___A002_Xc69057_X91a_bb1.ms


2019-10-26 14:40:34	INFO	mstransform::::casa	##########################################
2019-10-26 14:40:34	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-26 14:40:34	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/proc/2017.1.01045.S/science_goal.uid___A001_X1288_X10c4/group.uid___A001_X1288_X10c5/member.uid___A001_X1288_X10c6/calibrated/uid___A002_Xc69057_Xc5c.ms', outputvis='uid___A002_Xc69057_Xc5c_bb1.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='25', scan='', antenna='', correlation='', timerange='', intent='OBSERVE_TARGET#ON_SOURCE', array='', uvrange='', observation='', feed='', datacolumn='corrected', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='frequency', nchan=479, start='142633992684.47223Hz', width='3906467.6169433594Hz', nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='

...

2019-10-26 17:03:40	INFO	mstransform::::casa	##########################################
2019-10-26 17:03:40	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-26 17:03:40	INFO	mstransform::::casa	mstransform( vis='/Volumes/D1/proc/2017.1.01045.S/science_goal.uid___A001_X1288_X10c4/group.uid___A001_X1288_X10c5/member.uid___A001_X1288_X10c6/calibrated/uid___A002_Xc6c0d5_X57ea.ms', outputvis='uid___A002_Xc6c0d5_X57ea_bb4.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='31', scan='', antenna='', correlation='', timerange='', intent='OBSERVE_TARGET#ON_SOURCE', array='', uvrange='', observation='', feed='', datacolumn='corrected', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=True, chanaverage=False, chanbin=1, hanning=False, regridms=True, mode='frequency', nchan=479, start='154508695293.72736Hz', width='3906467.6171875Hz', nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='l

Before rmPointing()
1.2G	uid___A002_Xc6c0d5_X57ea_bb4.ms
After rmPointing()
453M	uid___A002_Xc6c0d5_X57ea_bb4.ms


2019-10-26 17:11:26	INFO	concat::::casa	##########################################
2019-10-26 17:11:26	INFO	concat::::casa	##### Begin Task: concat             #####
2019-10-26 17:11:26	INFO	concat::::casa	concat( vis=['uid___A002_Xc69057_X91a_bb4.ms', 'uid___A002_Xc69057_Xc5c_bb4.ms', 'uid___A002_Xc69057_X1293_bb4.ms', 'uid___A002_Xc6a3db_X430_bb4.ms', 'uid___A002_Xc6c0d5_X57ea_bb4.ms'], concatvis='bb4.ms', freqtol='', dirtol='', respectname=False, timesort=False, copypointing=True, visweightscale=[], forcesingleephemfield='' )
2019-10-26 17:11:26	INFO	concat::::casa	copying uid___A002_Xc69057_X91a_bb4.ms to bb4.ms
2019-10-26 17:11:27	INFO	concat::::casa	concatenating uid___A002_Xc69057_Xc5c_bb4.ms into bb4.ms
2019-10-26 17:11:27	INFO	MSConcat::concatenate 	Appending /Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/data/bx610/alma/2017.1.01045.S/uid___A002_Xc69057_Xc5c_bb4.ms to /Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/data/bx610/alma/2017.1.01045.S/bb4.ms
2019-10-26 17:11


run checkchflag on bb4.ms


query: DATA_DESC_ID==0
data shape:  (2, 479, 286407)

pol id: 0 variety: 2
10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 283657/286407
100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111111111111

2019-10-26 17:13:13	INFO	mstransform::::casa	##########################################
2019-10-26 17:13:13	INFO	mstransform::::casa	##### Begin Task: mstransform        #####
2019-10-26 17:13:13	INFO	mstransform::::casa	mstransform( vis='bb2.ms', outputvis='bb2.mfs.ms', createmms=False, separationaxis='auto', numsubms='auto', tileshape=[0], field='BX610', spw='0:0~477', scan='', antenna='', correlation='', timerange='', intent='', array='', uvrange='', observation='', feed='', datacolumn='data', realmodelcol=False, keepflags=False, usewtspectrum=False, combinespws=False, chanaverage=True, chanbin=478, hanning=False, regridms=False, mode='channel', nchan=-1, start=0, width=1, nspw=1, interpolation='linear', phasecenter='', restfreq='', outframe='', veltype='radio', preaverage=False, timeaverage=False, timebin='0s', timespan='', maxuvwdistance=0.0, docallib=False, callib='', douvcontsub=False, fitspw='', fitorder=0, want_cont=False, denoising_lib=True, nthreads=1, niter=1, disableparall

In [ ]:
142633016064.75403 144500299865.06604 3906451.4650878906 479